<a href="https://colab.research.google.com/github/charles14525/Emerging-technology-2/blob/main/Assignment_9_1_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [2]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

x_train, x_test = x_train.astype('float32') / 255.0, x_test.astype('float32') / 255.0

def baseline_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = baseline_cnn()
model.fit(x_train, y_train, epochs=5, validation_split=0.1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 44s 31ms/step - accuracy: 0.4092 - loss: 1.6391 - val_accuracy: 0.5724 - val_loss: 1.2250
Epoch 2/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 43s 30ms/step - accuracy: 0.5809 - loss: 1.1910 - val_accuracy: 0.6108 - val_loss: 1.1351
Epoch 3/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 79s 29ms/step - accuracy: 0.6343 - loss: 1.0401 - val_accuracy: 0.6252 - val_loss: 1.0853
Epoch 4/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 42s 30ms/step - accuracy: 0.6660 - loss: 0.9647 - val_accuracy: 0.6348 - val_loss: 1.0795
Epoch 5/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 81s 29ms/step - accuracy: 0.6960 - loss: 0.8719 - val_accuracy: 0.6328 - val_loss: 1.0849


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)


In [4]:
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train_std = (x_train - mean) / std
x_test_std = (x_test - mean) / std


In [5]:
datagen_zca = ImageDataGenerator(zca_whitening=True)
datagen_zca.fit(x_train)

augmented_iterator = datagen.flow(x_train, y_train, batch_size=64)

/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:1047: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn(


In [6]:
import os
save_dir = 'augmented_images'
os.makedirs(save_dir, exist_ok=True)

datagen.save_to_dir = save_dir
for _ in range(5):
    batch = next(datagen.flow(x_train, y_train, batch_size=1, save_to_dir=save_dir, save_format='png'))


In [ ]:
from tensorflow.keras.layers import Dropout, BatchNormalization

def improved_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = improved_cnn()
model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 164s 206ms/step - accuracy: 0.2789 - loss: 2.1512 - val_accuracy: 0.4388 - val_loss: 1.5579
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 150s 192ms/step - accuracy: 0.4233 - loss: 1.5957 - val_accuracy: 0.5369 - val_loss: 1.3154
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 154s 196ms/step - accuracy: 0.4733 - loss: 1.4728 - val_accuracy: 0.4577 - val_loss: 1.7239
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 152s 194ms/step - accuracy: 0.4984 - loss: 1.3977 - val_accuracy: 0.6009 - val_loss: 1.1129
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 146s 186ms/step - accuracy: 0.5204 - loss: 1.3342 - val_accuracy: 0.5758 - val_loss: 1.2143
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 208s 194ms/step - accuracy: 0.5466 - loss: 1.2815 - val_accuracy: 0.6323 - val_loss: 1.0277
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 205s 198ms/step - accuracy: 0.5597 - loss: 1.2592 - val_accuracy: 0.6307 - val_loss: 1.0556
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 148s 190ms/step - accuracy: 0.5698 -

In [ ]:
loss, acc = model.evaluate(x_test, y_test)
print(f'Final Model Accuracy: {acc:.3f}')

predictions = model.predict(x_test[:5])
print("Predicted class indices:", predictions.argmax(axis=1))
